In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
df = pd.read_csv('df_cleaned.csv')
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_clean,q2_clean,q1_tokenized,q2_tokenized,...,q1_count,q2_count,intersection_q1_q2_token,intersection_q1_q2_token_count,q1_stemmed_j,q2_stemmed_j,min_value,max_value,perc_count_min,perc_count_max
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,"['what', 'is', 'the', 'step', 'by', 'step', 'g...","['what', 'is', 'the', 'step', 'by', 'step', 'g...",...,14,12,"{'what', 'market', 'step', 'to', 'guide', 'sha...",11,step step guid invest share market india,step step guid invest share market,12,14,0.916667,0.785714
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,What is the story of Kohinoor KohiNoor Diamond,What would happen if the Indian government sto...,"['what', 'is', 'the', 'story', 'of', 'kohinoor...","['what', 'would', 'happen', 'if', 'the', 'indi...",...,8,13,"{'what', 'the', 'diamond', 'kohinoor'}",4,stori kohinoor kohinoor diamond,would happen indian govern stole kohinoor kohi...,8,13,0.500000,0.307692
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,"['how', 'can', 'i', 'increase', 'the', 'speed'...","['how', 'can', 'internet', 'speed', 'be', 'inc...",...,14,10,"{'how', 'speed', 'internet', 'can'}",4,increas speed internet connect use vpn,internet speed increas hack dn,10,14,0.400000,0.285714
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,Why am I mentally very lonely How can I solve it,Find the remainder when math2324math is divide...,"['why', 'am', 'i', 'mentally', 'very', 'lonely...","['find', 'the', 'remainder', 'when', 'math2324...",...,11,9,set(),0,mental lone solv,find remaind math2324math divid 2423,9,11,0.000000,0.000000
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,Which one dissolve in water quikly sugar salt ...,Which fish would survive in salt water,"['which', 'one', 'dissolve', 'in', 'water', 'q...","['which', 'fish', 'would', 'survive', 'in', 's...",...,13,7,"{'salt', 'which', 'water', 'in'}",4,one dissolv water quikli sugar salt methan car...,fish would surviv salt water,7,13,0.571429,0.307692


In [3]:
df.isnull().sum()

id                                 0
qid1                               0
qid2                               0
question1                          0
question2                          0
is_duplicate                       0
q1_clean                          12
q2_clean                           2
q1_tokenized                       0
q2_tokenized                       0
q1_nostop                          0
q2_nostop                          0
q1_stemmed                         0
q2_stemmed                         0
q1_count                           0
q2_count                           0
intersection_q1_q2_token           0
intersection_q1_q2_token_count     0
q1_stemmed_j                      78
q2_stemmed_j                      64
min_value                          0
max_value                          0
perc_count_min                    14
perc_count_max                     0
dtype: int64

In [4]:
df = df.dropna(subset=['q1_clean', 'q2_clean'], axis=0)

In [5]:
df.shape

(404273, 24)

In [6]:
!pip install gensim

In [7]:
import gensim
from gensim.models import Word2Vec

In [9]:
# Concatenate the tokenized words from 'q1_tokenized' and 'q2_tokenized'
sentences = df['q1_tokenized'].tolist() + df['q2_tokenized'].tolist()

# Train Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Save the trained model
model.save("word2vec.model")

In [10]:
# Define a function to calculate the average word embeddings
def calculate_average_embedding(tokens):
    embeddings = [model.wv[word] for word in tokens if word in model.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.vector_size)  # Return zero vector if no valid word embeddings found

# Apply the function to calculate average word embeddings for 'q1_tokenized'
df['word2vec_q1'] = df['q1_tokenized'].apply(calculate_average_embedding)

# Apply the function to calculate average word embeddings for 'q2_tokenized'
df['word2vec_q2'] = df['q2_tokenized'].apply(calculate_average_embedding)

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

batch_size = 1000  # Define the batch size

# Calculate the similarity values in batches
similarity_values = []
num_batches = len(df) // batch_size + 1  # Calculate the number of batches

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(df))
    batch_q1 = df['word2vec_q1'].tolist()[start_idx:end_idx]
    batch_q2 = df['word2vec_q2'].tolist()[start_idx:end_idx]
    
    batch_similarity = cosine_similarity(batch_q1, batch_q2).diagonal()
    similarity_values.extend(batch_similarity)

# Add the similarity values as a new column in the DataFrame
df['similarity_values'] = similarity_values

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report

In [16]:
X=df[['perc_count_min','perc_count_max','similarity_values']]
y=df['is_duplicate']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [58]:
model = LogisticRegression()
model.fit(X_train,y_train)

# # Calculate the training accuracy
train_accuracy = model.score(X_train, y_train)
print("Training Accuracy:", train_accuracy)

# # Calculate the testing accuracy
test_accuracy = model.score(X_test, y_test)
print("Testing Accuracy:", test_accuracy)

Training Accuracy: 0.6519674229634714
Testing Accuracy: 0.6483829076742317


In [59]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(X_train, y_train)

# Calculate the training accuracy
train_accuracy = model.score(X_train, y_train)
print("Training Accuracy:", train_accuracy)

# Calculate the testing accuracy
test_accuracy = model.score(X_test, y_test)
print("Testing Accuracy:", test_accuracy)

Training Accuracy: 0.7086309358168067
Testing Accuracy: 0.6987446663780842


In [18]:
# simple Network

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

2023-06-06 23:09:18.574247: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 23:09:18.794309: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [19]:
X_train = np.array(X_train)
X_train


array([[0.66666667, 0.66666667, 0.99844098],
       [0.16666667, 0.125     , 0.98933506],
       [1.        , 0.7       , 0.9995839 ],
       ...,
       [0.71428571, 0.71428571, 0.99957973],
       [0.        , 0.        , 0.99283606],
       [0.8       , 0.8       , 0.99958801]])

In [81]:
np.reshape(X_train, (X_train.shape[0], 3))

array([[0.66666667, 0.66666667, 0.9982692 ],
       [0.16666667, 0.125     , 0.9890976 ],
       [1.        , 0.7       , 0.99959415],
       ...,
       [0.71428571, 0.71428571, 0.99958426],
       [0.        , 0.        , 0.99178833],
       [0.8       , 0.8       , 0.99950725]])

In [24]:
# Convert the training data into numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

# Reshape the input arrays to have the correct shape
X_train = np.reshape(X_train, (X_train.shape[0], 3))

# Define the Neural Networks model
model = Sequential()
model.add(Dense(128, input_shape=(3,)))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, batch_size=10, validation_data=(X_test, y_test))

# Access the training and validation results
train_loss = history.history['loss']            # Training loss values
train_accuracy = history.history['accuracy']    # Training accuracy values
val_loss = history.history['val_loss']          # Validation loss values
val_accuracy = history.history['val_accuracy']  # Validation accuracy values

# Print the results
print("Training Loss:", train_loss)
print("Training Accuracy:", train_accuracy)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)



Epoch 1/10
32342/32342 [==============================] - 82s 3ms/step - loss: 0.5874 - accuracy: 0.6498 - val_loss: 0.5880 - val_accuracy: 0.6526
Epoch 2/10
32342/32342 [==============================] - 81s 2ms/step - loss: 0.5869 - accuracy: 0.6494 - val_loss: 0.5865 - val_accuracy: 0.6446
Epoch 3/10
32342/32342 [==============================] - 84s 3ms/step - loss: 0.5869 - accuracy: 0.6494 - val_loss: 0.5874 - val_accuracy: 0.6439
Epoch 4/10
32342/32342 [==============================] - 86s 3ms/step - loss: 0.5867 - accuracy: 0.6493 - val_loss: 0.5861 - val_accuracy: 0.6442
Epoch 5/10
32342/32342 [==============================] - 83s 3ms/step - loss: 0.5866 - accuracy: 0.6497 - val_loss: 0.5861 - val_accuracy: 0.6494
Epoch 6/10
32342/32342 [==============================] - 82s 3ms/step - loss: 0.5867 - accuracy: 0.6501 - val_loss: 0.5861 - val_accuracy: 0.6474
Epoch 7/10
32342/32342 [==============================] - 83s 3ms/step - loss: 0.5868 - accuracy: 0.6496 - val_loss: 0